In [1]:
from brain_hierarchy import AllenBrainHierarchy
from readCSV_helpers import *
from pls_helpers import PLS

And we'll need other python functions to easily read and manipulate data and make nice plots:

In [2]:
import pandas as pd
import numpy as np
import os

# import matplotlib.pyplot as plt
import plotly.graph_objects as go

## Load data

Below, you have to specify:
- ```animals_root```: Absolute path to the folder that contains the animal folders.
- ```group_1_name```: A meaningful string for Group 1.
- ```group_2_name```: A meaningful string for Group 2.
- ```group_1_names```: A list of names of the folders corresponding to animals in **Group 1** (e.g., Control group). Indeed, it is necessary to store the results in individual folders for each animal.
- ```group_2_names```: A list of names of the folders corresponding to animals in **Group 2** (e.g., Stress group).

In [3]:
# ####################################### SET PARAMETERS ####################################


animals_root = './data/QuPath_output/'
group_1_name = 'Control'
group_2_name = 'Stress'

data_input_path = './data/python_norm_output/'
data_output_path = './data/python_PLS_output/'
plots_output_path = './plots/python_output/'


# ###########################################################################################


if not(os.path.exists(data_output_path)):
    os.makedirs(data_output_path, exist_ok=True)

if not(os.path.exists(plots_output_path)):
    os.makedirs(plots_output_path, exist_ok=True)

In [4]:
group_1_results = pd.read_csv(os.path.join(data_input_path, f'results_cell_counts_{group_1_name}.csv'), sep='\t', header=[0, 1], index_col=[0,1])
group_2_results = pd.read_csv(os.path.join(data_input_path, f'results_cell_counts_{group_2_name}.csv'), sep='\t', header=[0, 1], index_col=[0,1])

group_1_names = {index[1] for index in group_1_results.index}
group_2_names = {index[1] for index in group_2_results.index}

group_1_tracers = list({cols[0] for cols in group_1_results.columns})
group_2_tracers = list({cols[0] for cols in group_2_results.columns})
assert len(group_1_tracers) == len(group_2_tracers) == 1, "The CSVs in input should have data for one tracer only."
assert group_1_tracers[0] == group_2_tracers[0], f"The data in the CSV of group 1 and 2 should refer to the same tracer. \
Instead group 1 CSV presents data for the tracer '{group_1_tracers[0]}', while group 2 CSV for '{group_2_tracers[0]}'."
tracer = group_1_tracers[0]

The data are stored in ```group_1_results``` and ```group_2_results```:

In [5]:
group_1_results

CFos                           
                         Density Percentage RelativeDensity
root     Control_17C  327.070292   1.000000        1.000000
         Control_18C   33.310720   1.000000        1.000000
         Control_19C  148.701106   1.000000        1.000000
grey     Control_17C  346.050729   0.942186        1.058032
         Control_18C   35.144334   0.941337        1.055046
...                          ...        ...             ...
VISrll6a Control_18C         NaN        NaN             NaN
         Control_19C         NaN        NaN             NaN
VISrll6b Control_17C         NaN        NaN             NaN
         Control_18C         NaN        NaN             NaN
         Control_19C         NaN        NaN             NaN

[3981 rows x 3 columns]

In [6]:
# from https://help.brain-map.org/display/api/Downloading+an+Ontology%27s+Structure+Graph
# StructureGraph id=1
path_to_allen_json = "./data/AllenMouseBrainOntology.json"

branches_to_exclude = ['retina','VS','grv','fiber tracts']
AllenBrain = AllenBrainHierarchy(path_to_allen_json, branches_to_exclude)

# Now, get the selected regions as a variable:
level = 6

AllenBrain.select_at_structural_level(level)
selected_regions = AllenBrain.get_selected_regions()
print(f'You selected {len(selected_regions)} regions at level {level}.')

AllenBrain.unselect_all()
AllenBrain.select_at_depth(level)
selected_regions = AllenBrain.get_selected_regions()
print(f'You selected {len(selected_regions)} regions at depth {level}.')

AllenBrain.unselect_all()
AllenBrain.select_from_csv("./data/AllenSummaryStructures.csv")
selected_regions = AllenBrain.get_selected_regions()
print(f'You selected {len(selected_regions)} Summary Structure regions.')

You selected 34 regions at level 6.
You selected 288 regions at depth 6.
You selected 315 Summary Structure regions.


# Partial Least Squares  

The analysis done below is taken from the tutorial written by [Krishnan et al.](https://www.sciencedirect.com/science/article/pii/S1053811910010074).  
Run the 2 cells below to get started.

In [7]:
# PLS
normalization = 'Density' # 'Density', 'Percentage' or 'RelativeDensity'
rank = 1

# Create a PLS object
# TODO: see what happens if analyze() does not include the NaN rows
cfosPLS = PLS(group_1_results, group_2_results, group_1_names, group_2_names, selected_regions, tracer, normalization)

# Show the matrix X
cfosPLS.X

,CLA,LA,BLA,BMA,PA,SCs,IC,MEV,SCO,MRN,...,SSp-ul,SSp-tr,SSp-un,CA1,CA2,CA3,ENTl,ENTm,VPL,VPM
Control_17C,395.448533,294.466332,316.294075,359.865743,351.548048,669.712088,478.719925,255.482971,133.169247,200.744565,...,587.020559,733.103276,745.640186,176.936454,116.416978,110.041172,193.961011,20.418112,134.439866,248.034979
Control_18C,31.023108,45.436144,19.962298,28.235998,27.700546,137.508600,157.947373,63.640862,19.948905,27.218506,...,42.107562,19.633106,31.154238,10.897004,28.385428,24.214791,10.705021,1.972721,17.188914,85.262299
Control_19C,29.899975,68.725095,115.028796,98.104357,23.632022,350.609005,360.789111,117.930471,147.416091,235.831520,...,167.514757,103.421312,180.726349,44.727107,124.717617,83.818751,51.631986,3.455328,189.658209,281.327028
Resilient_11R,451.755244,136.169534,104.162125,233.972748,96.277970,522.374723,341.734779,261.020876,431.809462,176.643230,...,493.315385,323.755896,471.155910,106.721410,84.453737,107.926086,38.735293,1.811846,125.839928,97.111342
Resilient_1R,185.900003,116.631952,147.846825,156.906756,101.355517,541.800537,344.439345,338.634331,469.932585,142.137001,...,214.713800,473.389172,372.784630,118.476449,65.456225,82.605920,73.438005,29.303930,85.506449,60.756518
Resilient_2R,118.590286,167.916156,160.780087,128.096352,186.991045,479.691840,341.794165,330.397935,223.933071,111.279033,...,45.306368,144.897532,72.152596,85.305916,59.928155,81.280236,57.693303,0.105648,17.447822,34.489283
Resilient_3R,47.581731,67.976036,48.393625,77.955622,51.369273,316.726915,215.134341,0.000000,161.762158,96.811877,...,51.057219,90.471237,94.057045,50.788129,15.919696,31.425939,5.536158,1.082529,38.553544,32.201888
Resilient_4R,438.063965,274.673650,228.323274,247.578993,345.021561,583.006274,528.174830,175.301933,262.976249,200.599906,...,264.600332,644.099110,389.567453,214.381271,168.586117,143.702316,223.942537,23.032348,59.429969,50.032363
Stress_10S,274.422365,226.602931,243.508946,282.600507,206.914564,728.799773,642.558885,580.199194,35.128019,231.188360,...,812.716144,515.422725,926.667245,105.970932,111.662208,130.464768,132.789279,258.000064,117.448675,195.987847
Stress_13S,279.261334,51.868285,116.931794,178.799268,1.253955,334.256585,463.738128,221.364705,944.187919,108.065983,...,205.319283,375.169819,333.746577,138.753631,127.113245,92.946233,0.323962,2.162408,56.071058,29.743349


In [8]:
# Show the matrix Y
pd.get_dummies(cfosPLS.y).rename(columns={0: group_2_name, 1: group_1_name})

,Stress,Control
Control_17C,0,1
Control_18C,0,1
Control_19C,0,1
Resilient_11R,1,0
Resilient_1R,1,0
Resilient_2R,1,0
Resilient_3R,1,0
Resilient_4R,1,0
Stress_10S,1,0
Stress_13S,1,0


The two matrices printed above (X and Y) illustrate the data on which the PLS is done.  
- ```X:``` The rows in this matrix are the mice. The columns in the matrix are the regions selected for analysis. The values in the matrix are the **normalized value of marked cells: in that region relative to the whole brain.** 
The normalization methods are either:
  + Density
  + Percentage (on the total number of detected marked cells outside of excluded regions)
  + RelativeDensity
- ```Y:``` The rows in this matrix are the mice. The columns in the matrix are the 2 groups. **A value in this matrix is 1 if the mice belongs to the specified group**.

In brief, PLS analyzes the relationship (correlation) between the columns of ```X``` and ```Y```. In our specific case, there will be 2 important outputs:
- **Salience scores**: Each brain region has a salience score. A high salience scores means that the brain region explains much of the correlation between ```X``` and ```Y```.  
- **Singular values**: These are the eigenvalues of the correlation matrix $R = Y^TX$.

## Random permutations to see whether we can differentiate signal from noise. 
Here, we randomly shuffle the group to which a mouse belongs, and calculate the singular values of the permuted dataset.  
From [Krishnan et al.](https://www.sciencedirect.com/science/article/pii/S1053811910010074):  
> The set of all the (permuted) singular values provides a sampling distribution of the singular values under the null hypothesis and, therefore can be used as a null hypothesis test.

*Note: running the cell below will take a few minutes.*

In [9]:
num_permutations = 5000
print(f'Randomly permuting singular values %d times ...'%num_permutations)
s,singular_values = cfosPLS.randomly_permute_singular_values(num_permutations)
print('Done!\n')

Randomly permuting singular values 5000 times ...
Done!



In [10]:
# TODO: move to Plotly

# Plot distribution of singular values
# plt.figure(figsize=(10,4))
# plt.hist(singular_values[:,0],bins=10)
# plt.axvline(cfosPLS.s[0], color='r')
# plt.xlabel('First singular value')
# plt.ylabel('Frequency')
# plt.legend([f'Experiment','Sampling distribution\nunder H0 (%d permutations)'%num_permutations])
# plt.show()

In [11]:
# Calculate p-value = Probability(experiment | H0)
p = (singular_values[:,0] > s[0]).sum() / num_permutations
print('p-value = '+str(p))

p-value = 0.5216


## Bootstrap to identify stable salience scores

Here, we use [bootstrapping](https://en.wikipedia.org/wiki/Bootstrapping_(statistics)) (= sampling of the mice in the dataset, with replacement) to get an estimate of which salience scores are stable.

From [Krishnan et al.](https://www.sciencedirect.com/science/article/pii/S1053811910010074):  
> When a vector of saliences is considered generalizable and is kept for further analysis, we need to identify its elements that are stable through resampling. In practice, the stability of an element is evaluated by dividing it by its standard error. [...] To estimate the standard errors, we create bootstrap samples which are obtained by sampling with replacement the observations in and (Efron and Tibshirani, 1986). A salience standard error is then estimated as the standard error of the saliences from a large number of these bootstrap samples (say 1000 or 10000). **The ratios are akin to a Z-score, therefore when they are larger than 2 the corresponding saliences are considered significantly stable.**

*Note: Running the cell below will take a few minutes.*

In [12]:
num_bootstrap = 5000
print(f'Bootstrapping salience scores {num_bootstrap} times...')
u_salience_scores,v_salience_scores = cfosPLS.bootstrap_salience_scores(rank, num_bootstrap)
print('Done!')

Bootstrapping salience scores 5000 times...
Done!


In [13]:
# Plot PLS salience scores
plot_threshold = 1.2 # Only brain regions with a salience higher than plot_threshold are shown. 2 is the significance threshold.
file_title = f'PLS_{tracer}_{normalization}.png'
tp, salient_regions = cfosPLS.plot_salience_scores(plot_threshold, plots_output_path, file_title, AllenBrain.brain_region_dict,
                              fig_width=1000, fig_height=2000)

In [14]:
salient_regions = salient_regions.reset_index()
salient_regions.columns = ['region', 'salience']
salient_regions['salience'] = salient_regions['salience'].abs()
salient_regions = salient_regions.sort_values(by='salience')
salient_regions.to_csv(os.path.join(data_output_path, 'salient_regions.csv'), sep=';', index=False)
salient_regions

,region,salience
29,MOs,1.217708
30,VISpl,1.266536
27,TRN,1.268608
9,AAA,1.319650
36,PAR,1.403644
...,...,...
58,LH,3.476487
59,SUM,3.534449
21,PVHd,3.608450
48,PVT,4.091691


In [15]:
pls_filename = f'PLS_{tracer}_{normalization}_salience_scores.csv'
save_results(v_salience_scores.rename(columns={0:'salience score'}), data_output_path, pls_filename)


! A results_python folder already existed in root. I am overwriting previous results!

Results are saved in ./data/python_PLS_output/

Done!


True

# Plot percentages

In [16]:
# In this case we wanted to normalize it based on the density, rather then the Percentage 
# I didn't modify the various labels in the plot as I was just focused on adapting the code to our dataset, rather then polishing it

threshold = 1e-2 # Only plot bars with value larger than threshold (1e-6, 1e-2, 3)
y_axis_label = 'region_names' # change this to 'acronym' to have acronyms on the y-axis

# Calculate mean values
group_1_df = pd.DataFrame(group_1_results[(tracer,normalization)].rename('cell counts'))
group_1_avg = group_1_df.reset_index().groupby('level_0').mean(numeric_only=True)
group_1_sem = group_1_df.reset_index().groupby('level_0').sem(numeric_only=True)

group_2_df = pd.DataFrame(group_2_results[(tracer,normalization)].rename('cell counts'))
group_2_avg = group_2_df.reset_index().groupby('level_0').mean(numeric_only=True)
group_2_sem = group_2_df.reset_index().groupby('level_0').sem(numeric_only=True)

# Determine which regions to plot  
mean_sum = group_1_avg + group_2_avg
#regs_to_plot = mean_sum[(mean_sum['cell counts']>threshold) & (mean_sum['cell counts'].notnull())].sort_values(by='cell counts').index.to_list()
regs_to_plot = cfosPLS.X.columns.to_list()

# y-axis, with seperate values for each region
y_axis_il, ticklabels = pd.factorize(group_1_df.loc[regs_to_plot].reset_index()['level_0'])
y_axis_bla, ticklabels = pd.factorize(group_2_df.loc[regs_to_plot].reset_index()['level_0'])
if(y_axis_label=='region_names'):
    ticklabels = [AllenBrain.brain_region_dict[reg] for reg in ticklabels]
     
fig = go.Figure()

# Barplot
fig.add_trace(go.Bar(
                     x = group_1_avg.loc[regs_to_plot]['cell counts'],
                     name = f'{group_1_name} mean',
                     error_x = dict(
                         type='data',
                         array=group_1_sem.loc[regs_to_plot]['cell counts']
                     )
              )
)

fig.add_trace(go.Bar(
                     x = group_2_avg.loc[regs_to_plot]['cell counts'],
                     name = f'{group_2_name} mean',
                     error_x = dict(
                         type='data',
                         array=group_2_sem.loc[regs_to_plot]['cell counts']
                     )
              )
)

fig.update_layout(barmode='group', colorway=['rgb(0,255,0)', 'rgb(255,0,0)'])

# Scatterplot (animals)
fig.add_trace(go.Scatter(
                    mode = 'markers',
                    y = y_axis_il - 0.2,
                    x = group_1_df.loc[regs_to_plot]['cell counts'],
                    name = f'{group_1_name} animals',
                    opacity=0.5,
                    marker=dict(
                        color='rgb(0,255,0)',
                        size=5,
                        line=dict(
                            color='rgb(0,0,0)',
                            width=1
                        )
                    )
              )
)

fig.add_trace(go.Scatter(
                    mode = 'markers',
                    y = y_axis_bla + 0.2,
                    x = group_2_df.loc[regs_to_plot]['cell counts'],
                    name = f'{group_2_name} animals',
                    opacity=0.5,
                    marker=dict(
                        color='rgb(255,0,0)',
                        size=5,
                        line=dict(
                            color='rgb(0,0,0)',
                            width=1
                        )
                    )
              )
)

# Figure title
title = ''
if normalization == 'RelativeDensity':
    title = f'[#{tracer} / area] / [{tracer} (brain) / area (brain)].'
elif normalization == 'Density':
    title = f'[#{tracer} / area]'
elif normalization == 'Percentage':
    title = f'[#{tracer} / brain]'

# Update layout
fig.update_layout(
    title = title,
    yaxis = dict(
        tickmode = 'array',
        tickvals = np.arange(0,len(regs_to_plot)),
        ticktext = ticklabels
    ),
    xaxis=dict(
        title = f'{tracer} density (relative to brain)'
    ),
    width=900, height=5000,
    hovermode="x unified",
    yaxis_range = [-1,len(regs_to_plot)+1]
)

fig.show()

# Save figure as PNG
file_title = 'barplot_' + tracer + '_' + normalization + 'CvS.png'
fig.write_image(os.path.join(plots_output_path, file_title))